## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [ ]:
import pickle
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

In [ ]:
test_img = mpimg.imread('camera_cal/calibration3.jpg')
plt.imshow(test_img)

# Calibrate camera and test undistort

In [ ]:
def turn_to_gray(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray

def find_corner_points(img, grid_shape):
    ret, corners = cv2.findChessboardCorners(img, grid_shape, None)
    return ret, corners
    
def calibrate_camera(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[:-1], None, None)
    return ret, mtx, dist, rvecs, tvecs
    
def cal_undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [ ]:
grid_shape = (9, 6)
objp = np.zeros((grid_shape[1]*grid_shape[0],3), np.float32)
objp[:,:2] = np.mgrid[0:grid_shape[0],0:grid_shape[1]].T.reshape(-1,2)
objpoints = []
imgpoints = []
imgs = glob.glob('./camera_cal/calibration*.jpg')

for fname in imgs:
    img = mpimg.imread(fname)
    ret, corners =  find_corner_points(turn_to_gray(img), grid_shape)
    
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        img = cv2.drawChessboardCorners(img, grid_shape, corners, ret)
        plt.figure()        
        plt.imshow(img)
        plt.suptitle(fname)

In [ ]:
ret, mtx, dist, rvecs, tvecs = calibrate_camera(test_img, objpoints, imgpoints)
undistorted = cal_undistort(test_img, mtx, dist)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(undistorted)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Perspective shift

In [ ]:
def warp_perspective(img, src, dst):
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped, M

In [ ]:
ret, corners =  find_corner_points(turn_to_gray(undistorted), grid_shape)
offset = 100
img_size = (undistorted.shape[1], undistorted.shape[0])

src = np.float32([corners[0][0], corners[grid_shape[0]-1][0], corners[-1][0], corners[-grid_shape[0]][0]])
dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                             [img_size[0]-offset, img_size[1]-offset], 
                             [offset, img_size[1]-offset]])

undistorted = cv2.drawChessboardCorners(undistorted, grid_shape, corners, ret)

warped, M = warp_perspective(test_img, src, dst)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(undistorted)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(warped)
ax2.set_title('Perpective warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
straight_lines = mpimg.imread('test_images/straight_lines1.jpg')

print(straight_lines.shape)
pts1 = np.array([[580,450],[700,450],[1235,720],[45,720]], np.int32)
pts2 = np.array([[145,0],[1135,0],[1135,720],[145,720]], np.int32)
src = np.float32(pts1)
dst = np.float32(pts2)
pts1 = pts1.reshape((-1,1,2))
pts2 = pts2.reshape((-1,1,2))

warped, M = warp_perspective(straight_lines, src, dst)
cv2.polylines(straight_lines,[pts1],True,(255,0,0), thickness=2)
cv2.polylines(warped,[pts2],True,(255,0,0), thickness=2)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(straight_lines)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(warped)
ax2.set_title('Perpective warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
straight_lines = mpimg.imread('test_images/straight_lines2.jpg')
warped, M = warp_perspective(straight_lines, src, dst)
cv2.polylines(straight_lines,[pts1],True,(255,0,0), thickness=2)
cv2.polylines(warped,[pts2],True,(255,0,0), thickness=2)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(straight_lines)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(warped)
ax2.set_title('Perpective warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Tresholding

In [ ]:
def abs_sobel_thresh(gray, orient='x', sobel_kernel=3, thresh=(0, 255)):
    x_orient = 1 if orient == 'x' else 0
    y_orient = 0 if orient == 'x' else 1
    sobel = cv2.Sobel(gray, cv2.CV_64F, x_orient, y_orient)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary

def mag_thresh(gray, sobel_kernel=3, mag_thresh=(0, 255)):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scaled_gradmag = np.uint8(255*gradmag/np.max(gradmag))
    mag_binary = np.zeros_like(scaled_gradmag)
    mag_binary[(scaled_gradmag >= mag_thresh[0]) & (scaled_gradmag <= mag_thresh[1])] = 1
    return mag_binary

def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    grad_direction = np.arctan2(abs_sobely, abs_sobelx)
    dir_binary =  np.zeros_like(grad_direction)
    dir_binary[(grad_direction >= thresh[0]) & (grad_direction <= thresh[1])] = 1
    return dir_binary

In [ ]:
test_img_path = 'test_images/test1.jpg'
test_img = mpimg.imread(test_img_path)
test_img_gray = turn_to_gray(test_img)
result_x = abs_sobel_thresh(test_img_gray, orient='x', sobel_kernel=20, thresh=(20, 100))
result_y = abs_sobel_thresh(test_img_gray, orient='y', sobel_kernel=20, thresh=(20, 100))
combined = np.zeros_like(test_img_gray)
combined[((result_x == 1) & (result_y == 1))] = 1

f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(result_x, cmap='gray')
ax2.set_title('Binary X', fontsize=50)
ax3.imshow(result_y, cmap='gray')
ax3.set_title('Binary Y', fontsize=50)
ax4.imshow(combined, cmap='gray')
ax4.set_title('Binary Combined X,Y', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
test_img = mpimg.imread(test_img_path)
test_img_gray = turn_to_gray(test_img)
result_mag = mag_thresh(test_img_gray, sobel_kernel=15, mag_thresh=(20, 100))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(result_mag, cmap='gray')
ax2.set_title('Binary Mag', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
test_img = mpimg.imread(test_img_path)
test_img_gray = turn_to_gray(test_img)
result_dir = dir_threshold(test_img_gray, sobel_kernel=15, thresh=(0.3, 1.7))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(result_dir, cmap='gray')
ax2.set_title('Binary Dir', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:

combined = np.zeros_like(result_x)
combined[((result_x == 1) & (result_y == 1)) | ((result_mag == 1) & (result_dir == 1))] = 1

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(combined, cmap='gray')
ax2.set_title('Binary Combined', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Color Thresholding

In [ ]:
test_img = mpimg.imread(test_img_path)

plt.imshow(test_img)
plt.suptitle("Original Image")

red = test_img[:, :, 0]

plt.figure()
plt.imshow(red, cmap='gray')
plt.suptitle("Red Channel")

green = test_img[:, :, 1]

plt.figure()
plt.imshow(green, cmap='gray')
plt.suptitle("Green Channel")


blue = test_img[:, :, 2]

plt.figure()
plt.imshow(blue, cmap='gray')
plt.suptitle("Blue Channel")


gray = turn_to_gray(test_img)

plt.figure()
plt.imshow(gray, cmap='gray')
plt.suptitle("Gray Channel")

hls = cv2.cvtColor(test_img, cv2.COLOR_RGB2HLS)

hls_h = hls[:, :, 0]

plt.figure()
plt.imshow(hls_h, cmap='gray')
plt.suptitle("HLS Hue Channel")


hls_l = hls[:, :, 1]

plt.figure()
plt.imshow(hls_l, cmap='gray')
plt.suptitle("HLS L Channel")

hls_s = hls[:, :, 2]

plt.figure()
plt.imshow(hls_s, cmap='gray')
plt.suptitle("HLS S Channel")

hsv = cv2.cvtColor(test_img, cv2.COLOR_RGB2HSV)

hsv_h = hsv[:, :, 0]

plt.figure()
plt.imshow(hsv_h, cmap='gray')
plt.suptitle("HSV H Channel")

hsv_s = hsv[:, :, 1]

plt.figure()
plt.imshow(hsv_s, cmap='gray')
plt.suptitle("HSV S Channel")

hsv_v = hsv[:, :, 2]

plt.figure()
plt.imshow(hsv_v, cmap='gray')
plt.suptitle("HSV V Channel")

In [ ]:
def hls_select_S(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    binary_output = np.zeros_like(S)
    binary_output[(S > thresh[0]) & (S <= thresh[1])] = 1
    return binary_output

def hls_select_H(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    H = hls[:,:,2]
    binary_output = np.zeros_like(H)
    binary_output[(H > thresh[0]) & (H <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

def hsv_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    V = hls[:,:,2]
    binary_output = np.zeros_like(V)
    binary_output[(V > thresh[0]) & (V <= thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output


In [ ]:
test_img = mpimg.imread(test_img_path)
result_s = hls_select_S(test_img, thresh=(170, 255))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(result_s, cmap='gray')
ax2.set_title('Binary S', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
test_img = mpimg.imread(test_img_path)
result_h = hls_select_H(test_img, thresh=(100, 255))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(result_h, cmap='gray')
ax2.set_title('Binary H', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
test_img = mpimg.imread(test_img_path)
result_v = hsv_select(test_img, thresh=(170, 255))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(result_v, cmap='gray')
ax2.set_title('Binary V', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
combined = np.zeros_like(result_x)
combined[((result_x == 1) | (result_s == 1))] = 1

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(combined, cmap='gray')
ax2.set_title('Binary Combine SobelX & S', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

# Lane Finding

In [ ]:
warped = combined
warped, M = warp_perspective(warped, src, dst)
cv2.polylines(test_img,[pts1],True,(255,0,0), thickness=2)

out_img = np.dstack((warped, warped, warped))*255

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(out_img, cmap='gray')
ax2.set_title('Perpective warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
def find_lane_lines(binary_warped):    
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    return histogram, left_fit, right_fit, nonzeroy, nonzerox, left_lane_inds, right_lane_inds

In [ ]:
histogram, left_fit, right_fit, nonzeroy, nonzerox, left_lane_inds, right_lane_inds = find_lane_lines(warped)

plt.plot(histogram)
axes = plt.gca()
axes.set_xlim([0,1280])
axes.set_ylim([0,500])

In [ ]:
ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

In [ ]:
nonzero = warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
margin = 100
left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

In [ ]:
out_img = np.dstack((warped, warped, warped))*255
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

In [ ]:
y_eval = np.max(ploty)
left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
print(left_curverad, right_curverad)

In [ ]:
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# Fit new polynomials to x,y in world space
left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
# Calculate the new radii of curvature
left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
# Now our radius of curvature is in meters
print(left_curverad, 'm', right_curverad, 'm')

In [ ]:
def draw_detected_lane(binary_warped, original, left_fitx, right_fitx, ploty):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp, M = warp_perspective(color_warp, dst, src) 
    # Combine the result with the original image
    result = cv2.addWeighted(original, 1, newwarp, 0.3, 0)
    return result

In [ ]:
test_img = mpimg.imread(test_img_path)
result = draw_detected_lane(warped, test_img, left_fitx, right_fitx, ploty)
plt.imshow(result)

In [ ]:
def process_image(img):
    gray = turn_to_gray(img)
    binary_x = abs_sobel_thresh(gray, orient='x', sobel_kernel=20, thresh=(20, 100))
    binary_s = hls_select_S(img, thresh=(170, 255))
    #binary_h = hls_select_H(img, thresh=(170, 255))
    binary_combined = np.zeros_like(binary_s)
    binary_combined[((binary_s == 1) | (binary_h == 1))] = 1
    binary_warped, M = warp_perspective(binary_combined, src, dst)
    histogram, left_fit, right_fit, nonzeroy, nonzerox, left_lane_inds, right_lane_inds = find_lane_lines(binary_warped)
    
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    result = draw_detected_lane(binary_warped, img, left_fitx, right_fitx, ploty)
    return result

In [ ]:
output = 'output_images/project.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(output, audio=False)

In [ ]:
HTML("<video width=\"960\" height=\"540\" controls><source src=\"{0}\"></video>".format(output))